## Data Preparation

In [8]:
import os
from dotenv import load_dotenv
from roboflow import Roboflow

load_dotenv(override=True)

ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')
if not ROBOFLOW_API_KEY:
    raise ValueError("ROBOFLOW_API_KEY not found in environment variables")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
# project = rf.workspace("jet-znmu4").project("help_me")
project = rf.workspace("ia-4afep").project("voitures-mmcgl")
version = project.version(1)
dataset = version.download("coco")

print("Dataset downloaded successfully")

loading Roboflow workspace...
loading Roboflow project...
Exporting format coco in progress : 85.0%
Version export complete for coco format



Extracting Dataset Version Zip to voitures-1 in coco:: 100%|██████████| 80/80 [00:00<00:00, 1437.46it/s]

Dataset downloaded successfully


## Training YOLOv8

In [10]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')
# Or load a custom model
# model = YOLO('path/to/best.pt')  

# Train the model
results = model.train(
    data=r"voitures-1",
    epochs=100,
    imgsz=640,
    batch=16,
    workers=8,
    device='cpu'
)

# Validate the model
metrics = model.val()

New https://pypi.org/project/ultralytics/8.3.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.21 🚀 Python-3.12.7 torch-2.5.0 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=voitures-1, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

RuntimeError: Dataset 'voitures-1' error ❌ [Errno 21] Is a directory: 'voitures-1'

## Deploying to Roboflow

In [ ]:
from roboflow import Roboflow

# Initialize Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Create or access your project
project = rf.workspace("railguard").project("railstuff")

# Upload your trained model
project.version(1).deploy(
    model_type="yolov8",
    model_path="path/to/best.pt",
    confidence=0.5
)

In [ ]:
from roboflow import Roboflow
import cv2

# Initialize Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Get your model
model = rf.workspace("YOUR_WORKSPACE").project("YOUR_PROJECT").version(1).model

# Make predictions
results = model.predict(
    "path/to/image.jpg",
    confidence=40,
    overlap=30
).json()

# Process results
for prediction in results['predictions']:
    # Extract prediction details
    class_name = prediction['class']
    confidence = prediction['confidence']
    x = prediction['x']
    y = prediction['y']
    width = prediction['width']
    height = prediction['height']
    
    print(f"Found {class_name} with {confidence*100:.2f}% confidence")

In [1]:
from roboflow import Roboflow

# Initialize and load model
rf = Roboflow(api_key="OqH8tRmlNXst2MdXI893")
project = rf.workspace("manas-mtp").project("mtp-vehicles")
model = project.version(27).model

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Import required libraries
from roboflow import Roboflow
import cv2
import matplotlib.pyplot as plt
import requests
from PIL import Image
import io
import numpy as np

# Download a sample image of a car on train tracks
def get_sample_image():
    # Using a sample image URL - this is a placeholder URL, it will be saved locally
    url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/GAZ-13_Chaika_draisine.jpg/991px-GAZ-13_Chaika_draisine.jpg"
    
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    img.save("car_on_tracks.jpg")
    return "car_on_tracks.jpg"

# Function to display image in notebook
def display_image_with_predictions(image_path, predictions):
    print(f">>>>>>>>>> {predictions}")
    # Read image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib
    
    # Create figure with larger size
    plt.figure(figsize=(15, 10))
    
    # Draw predictions
    for prediction in predictions['predictions']:
        # Get prediction details
        class_name = prediction['class']
        confidence = prediction['confidence']
        x = prediction['x']
        y = prediction['y']
        width = prediction['width']
        height = prediction['height']
        
        # Calculate bounding box coordinates
        x1 = int(x - width/2)
        y1 = int(y - height/2)
        x2 = int(x + width/2)
        y2 = int(y + height/2)
        
        # Draw rectangle
        plt.gca().add_patch(plt.Rectangle((x1, y1), width, height,
                                        fill=False, color='red', linewidth=2))
        
        # Add label
        plt.text(x1, y1-10, f'{class_name}: {confidence:.2f}',
                color='red', fontsize=12, backgroundcolor='white')
    
    # Display image
    plt.imshow(image)
    plt.axis('off')
    plt.title('Detected Objects')
    plt.show()
    
    # Print predictions
    print("\nDetected objects:")
    for pred in predictions['predictions']:
        print(f"- {pred['class']} (Confidence: {pred['confidence']:.2f})")

# Main execution
def main():
    # Get sample image
    image_path = get_sample_image()
    
    # Initialize Roboflow and load model
    rf = Roboflow(api_key="OqH8tRmlNXst2MdXI893")
    project = rf.workspace("manas-mtp").project("mtp-vehicles")
    model = project.version(27).model
    
    # Make prediction
    predictions = model.predict(image_path, confidence=40, overlap=30).json()
    
    # Display results
    display_image_with_predictions(image_path, predictions)

# Run in Jupyter notebook
if __name__ == "__main__":
    main()

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x10bc78a40>

In [3]:
from roboflow import Roboflow

# Source model
source_rf = Roboflow(api_key="OqH8tRmlNXst2MdXI893")
source_project = source_rf.workspace("manas-mtp").project("mtp-vehicles")
source_model = source_project.version(27)

# Your workspace
your_rf = Roboflow(api_key="OqH8tRmlNXst2MdXI893")
your_workspace = your_rf.workspace("railguard")
your_project = your_workspace.project("railstuff")

# Download the model weights for YOLOv8
# This will save the model to your local machine, where you can load and deploy it as needed
model_path = source_model.download("yolov8")
print(f"Model downloaded to: {model_path}")

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to MTP-Vehicles-27 in yolov8:: 100%|██████████| 5828/5828 [00:03<00:00, 1732.28it/s]


Model downloaded to: <roboflow.core.dataset.Dataset object at 0x131ff1880>


In [6]:
import os
import zipfile

# Your workspace
your_rf = Roboflow(api_key="OqH8tRmlNXst2MdXI893")
your_workspace = your_rf.workspace("railguard")
your_project = your_workspace.project("railstuff")

# Create a zip file of the model artifacts
model_zip_path = os.path.join(os.path.dirname(model_path), "yolov8-model.zip")
with zipfile.ZipFile(model_zip_path, "w") as zip_file:
    for root, _, files in os.walk(model_path):
        for file in files:
            zip_file.write(os.path.join(root, file))

# Upload the zipped model
your_project.upload(model_zip_path, model_name="yolov8-model", model_type="yolov8")
print("Model uploaded successfully!")

loading Roboflow workspace...
loading Roboflow project...


TypeError: expected str, bytes or os.PathLike object, not Dataset